# Compound Bow


## Imports

In [ ]:
from sympy import *
from sympy.physics.mechanics import *
from IPython.display import display

## Symbols

The retract distance for the bow is given by $x$, and the forward speed by $v$. 
Thus, the single generalized coordinate is $x$, and the corresponding generalized speed is $v$. 
Because the positive direction of the two is opposite, we define the kinematic equation $\dot{x} = -v$, or by the convention used in `sympy`, $\dot{x}+v=0$.

In [ ]:
x, v = dynamicsymbols('x, v')
kd = [diff(x) + v]

We will define the origin of the model as a point at the grip of the bow. 
The line is initially set back a constant distance $x_0$ from this point, at which point forces are in equilibrium.

In [ ]:
x0 = symbols('x_0')

The limbs of the bow will be represented by rigid bodies with a rotational spring attached at fixed positions $(x_l, y_l)$ and $(x_l, -y_l)$ relative to the origin. 


In [ ]:
xl, yl = symbols('x_l, y_l')

The extent of the limb's angular displacement relative to $\theta_0$ is $\theta$, which is measured relative to the undeflected angle $\theta_0$ which is measured relative to the horizontal. 

In [ ]:
theta = dynamicsymbols('\\theta')
theta0 = symbols('\\theta')

The length of each limb is $l$. 

In [ ]:
l = symbols('l')

The mass and moment of inertia for each limb are $m_l$ and $I_l$, respectively.

In [ ]:
ml, Il = symbols('m_l, I_l')

The effective angular stiffness of each limb is $k$. 

In [ ]:
k = symbols('k')

The inner and outer radii at the cam are defined as $r_i$ and $r_o$, respectively. 
These are initially defined as constants, though they may alternately be defined as functions of angular displacement for non-circular geometries.

In [ ]:
ri, ro = symbols('r_i, r_o')

The angular displacement for the cams relative to their initial angle is $\phi$.

In [ ]:
phi = dynamicsymbols('\\phi')

The angle for the inner cam contact point relative to the local horizontal axis is denoted $\delta$.
The angle for the outer cam contact point relative to local horizontal is denoted $\gamma$.

In [ ]:
delta, gamma = symbols('\\delta, \\gamma')

## Assumptions

A basic assumption to be applied throughout this model is that there is symmetry in the upper and lower sections of the bow.
To reflect this symmetry, points in the upper section will use a character, and the lower section will use the same character with a prime symbol.
Translational and angular displacements are assumed equal and opposite in the upper and lower sections, which will be represented by using the negative $y$ component of the rotated frame from the upper section when representing coordinates in the lower section.

## Frames

The inertial reference frames is defined with basis vectors $\hat{\boldsymbol{i}}$, $\hat{\boldsymbol{j}}$, and $\hat{\boldsymbol{k}}$.

In [ ]:
latexs = [r'\hat{\boldsymbol{' + ax + '}}' for ax in ('i', 'j', 'k')]
inertial = ReferenceFrame('N', latexs=latexs)

The limb orientations are defined using the frame $A$.

In [ ]:
A = inertial.orientnew('A', 'Axis', [theta0+theta, inertial.z])
# A_ = inertial.orientnew('A^\\prime', 'Axis', [-theta0-theta, inertial.z])
#display(A.dcm(inertial))

The cam orientations are defined using frames $B$.

In [ ]:
B = inertial.orientnew('B', 'Axis', [phi, inertial.z])
# B_ = inertial.orientnew('B^\\prime', 'Axis', [-phi, inertial.z])

## Points

The origin is designated $O$, and has zero velocity in the inertial frame.

In [ ]:
origin = Point('O')
origin.set_vel(inertial, Vector(0))

The point representing the position of the retracted line is designated $p$. 

In [ ]:
p = origin.locatenew('p', (x+x0)*inertial.x)
p.set_vel(inertial, -v*inertial.x)

The centers of rotation of the limbs are defined as points designated $a$ and $A^\prime$.

In [ ]:
a = origin.locatenew('a', xl*inertial.x + yl*inertial.y)
a_ = origin.locatenew('a^\\prime', xl*inertial.x - yl*inertial.y)

The centers of rotation of the cams are located at points designated $b$ and $b^\prime$.

In [ ]:
b = a.locatenew('b', l*A.x)
b_ = a_.locatenew('b^\\prime', -l*A.x)
display(b.pos_from(origin))
display(b_.pos_from(origin))

The points at which the lines contact the inner cam are designated $c$ and $c^\prime$.

In [ ]:
c = b.locatenew('c', -ri*cos(delta)*inertial.x - ri*sin(delta)*inertial.y)
c_ = b_.locatenew('c^\\prime', -ri*cos(delta)*inertial.x + ri*sin(delta)*inertial.y)

In [ ]:
display(c_.pos_from(origin))

The points at which the lines contact the outer cams are designated $d$ and $d^\prime$.
Similar to the inner cam, if the cam is circular, then a right triangle is formed, in this case with the hypotenuse formed between the contact point and the aft line where the archer holds their finger. 

In [ ]:
d = b.locatenew('d', ro*cos(gamma)*inertial.x + ro*sin(gamma)*inertial.y)
d_ = b_.locatenew('d^\\prime', ro*cos(gamma)*inertial.x + ro*sin(gamma)*inertial.y)
display(d_.pos_from(origin))

## Functions

For circular cams, we can assume the line is tangent to the edge of the circle, in this case the position of the contact point can be determined from a right triangle with the distance between $b$ and $b^\prime$ forming the hypotenuse, and the circle radius forming one edge.
For non-circular cams, a separate function is required to determine this contact point.

In [ ]:
rho_b = b.pos_from(b_).dot(inertial.y)
delta_fcn = asin(ri/rho_b)
display(delta_fcn)